HHU Deep Learning, SS2022/23, 09.06.2023, Prof. Dr. Markus Kollmann

Lecturers and Tutoring is done by Tim Kaiser, Nikolas Adaloglou and Felix Michels.

# Assignment 10 - Contrastive Language-Image Pre-training for unsupervised out-of-distribution detection

Copyright © 2023 Nikolas Adaloglou, Tim Kaiser and Felix Michels


## Contents

1. Basic imports
2. Get the visual features of the CLIP model
3. Compute the k-NN similarity as the OOD score
4. Compute MSP using the text encoder and the label names
5. Linear probing on the pseudolabels
6. Mahalanobis distance as OOD score
7. Mahalanobis distance using the real labels without linear probing
8. K-means clusters combined with Mahalanobis distance

---

## Overview
We will apply the learned representations from Contrastive Language-Image Pretrained (CLIP) on the downstream task of out-of-distribution detection.

`Note`: I used the pretrained models from open_clip_torch, you can install it with `!pip install open_clip_torch`

We will be using the model 'convnext_base_w' pretrained on 'laion2b_s13b_b82k' throughout this tutorial.

Info and examples on how to use CLIP models for inference is provided in [openclip](https://github.com/mlfoundations/open_clip#usage)

- [Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/abs/2103.00020)
- [Contrastive Language-Image Pretrained (CLIP) Models are Powerful Out-of-Distribution Detectors](https://arxiv.org/abs/2303.05828)



# Part I. Basic imports

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import torch
import open_clip
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import Subset, DataLoader, Dataset

out_dir = Path('./features/').resolve()
out_dir.mkdir(parents=True, exist_ok=True)
# Local import
from utils import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Helper function
def auroc_score(score_in, score_out):
    if type(score_in) == torch.Tensor:
        score_in = score_in.cpu().numpy()
        score_out = score_out.cpu().numpy()
    labels = np.concatenate((np.ones_like(score_in), np.zeros_like(score_out)))
    return roc_auc_score(labels, np.concatenate((score_in, score_out))) * 100

# Part II. Get the visual features of the CLIP model

- We will use `CIFAR100` as the in-distribution, and `CIFAR10` as the out-distribution.
- When you are only loading the visual CLIP backbone, you must remove the final linear layer that projects the features to the shared feature space of the image-text encoder.
- Load the data, compute the visual features and save them in the `features` folder.
- For the in-distribution you need both the train and test split, while for the out-distribution, we will only use the validation split.


### Optional structure

```python
def load_datasets(indist="CIFAR100", ood="CIFAR10", batch_size=256, tranform=None):
    # ....
    return indist_train_loader, indist_test_loader, ood_loader

# visual is a boolean that controls whether the visual backbone is only returned or the whole CLIP model
def get_model(visual, name, pretrained)
    # .....
    if visual:
        return backbone, preprocess
    return model, preprocess, tokenizer
    
# Load everything .......

feats, labels = get_features(backbone, dl, device)
# Save features 
# ....
```

In [ ]:
### START CODE HERE ### (≈ 31 lines of code)

### END CODE HERE ###

# feature test
for name, N in [('cifar100_train', 50000), ('cifar100_test', 10000), ('cifar10_test', 10000)]:
    feats = torch.load(f'features/{name}_feats.pt')
    labels = torch.load(f'features/{name}_labels.pt')
    assert feats.shape == (N, 1024)
    assert labels.shape == (N,)
print('Success!')

# Part III. Compute the k-NN similarity as the OOD score

- For each test image of in and out distribution compute the top-1 cosine similarity and use it as OOD score.
- Report the resulting AUROC score.
- Note: Use the image features and not the images!

In [ ]:
@torch.no_grad()
def OOD_classifier_knn(train_features, test_features, k=1):
    ### START CODE HERE ### (≈ 13 lines of code)
    
    ### END CODE HERE ###
    return cos_sim 

# load the computed features and compute scores
### START CODE HERE ### (≈ 5 lines of code)
indist_train = ...
indist_test = ...
ood_test = ...
score_in = ...
score_out =  ...
### END CODE HERE ###
print(f'CIFAR100-->CIFAR10 AUROC: {auroc_score(score_in, score_out):.2f}')

### Expected result

```
CIFAR100-->CIFAR10 AUROC: 83.55
```


# Part IV. Compute MSP using the text encoder and the label names

We will now consider the case where the in-distribution label names are available.

Your task is to apply zero-shot classification and get the maximum softmax probability (MSP) as the OOD score.

In short:
- compute image and text embeddings
- compute the image-test similarity matrix (logits)
- apply softmax to the logits for each image to get a probability distribution of the classes.
- compute maximum softmax probability (MSP)

- `Note`: After loading the saved image features you need to apply the linear projection layer from the visual backbone of CLIP

In [ ]:
### optional to use
def compute_logits(model, text_embs, img_embs, device):
    ### START CODE HERE ### (≈ 5 lines of code)
    
    ### END CODE HERE ###
    return logits

### optional to use
def compute_text_embeds(model, class_tokens):
    ### START CODE HERE ### (≈ 3 lines of code)
    
    ### END CODE HERE ###
    return text_embs

def compute_msp(label_names, model, class_tokens, indist_test, ood_test, device):
    ### START CODE HERE ### (≈ 7 lines of code)
   
    ### END CODE HERE ###
    return score_in, score_out
        

# Load model and features
### START CODE HERE ### (≈ 4 lines of code)
indist_test = ...
ood_test = ...
model, preprocess, tokenizer = ...
model = ...
### END CODE HERE ###

### Provided 
label_names = torchvision.datasets.CIFAR100(root='../data', train=True, download=True).classes
prompts = ['an image of a ' + lab.replace('_', ' ') for lab in label_names]
class_tokens = tokenizer(label_names).to(device)
score_in, score_out = compute_msp(label_names, model, class_tokens, indist_test, ood_test, device)
print(f'CIFAR100-->CIFAR10 AUROC: {auroc_score(score_in, score_out):.2f}')

### Expected result

```
CIFAR100-->CIFAR10 AUROC: 76.38
```


# Part V. Linear probing on the pseudolabels

- Your task is to train a linear layer using the CLIP pseudolabels as targets.
- The pseudolabels are the argmax of the logits computed above, i.e., take the class with the maximum probability as the class label

In [ ]:
def compute_score_probe_msp(lin_layer, indist_loader, ood_loader, device):
    """
    Computes the MSP scores for a linear layer for both in- and out- distribution.
    """
    ### START CODE HERE ### (≈ 4 lines of code)
    
    ### END CODE HERE ###
    return score_in, score_out

### START CODE HERE ### (≈ 17 lines of code) 
# get CLIP model
# get text embeds from label names

# load features

# compute CLIP logits of image features based on text encoder

# get target pseudo labels from CLIP logits

# create dataset and dataloaders for linear probing

### END CODE HERE ###

# The code below is provided based on our implementation. Optional to use!
# Run linear probing
embed_dim = train_dataset[0][0].shape[0]
lin_layer = nn.Linear(embed_dim, 100).to(device)
optimizer = torch.optim.Adam(lin_layer.parameters(), lr=1e-3)
num_epochs = 20
dict_log = linear_eval(lin_layer, optimizer, num_epochs, train_loader, val_loader, device)
# compute MSP scores
lin_layer = load_model(lin_layer, "CLIP_best_max_train_acc.pth")
ood_dataset = torch.utils.data.TensorDataset(ood_test, torch.zeros(ood_test.shape[0], dtype=torch.long))
ood_loader = torch.utils.data.DataLoader(ood_dataset, batch_size=128, shuffle=False, drop_last=False)
score_in, score_out = compute_score_probe_msp(lin_layer, val_loader, ood_loader, device)
print(f'CIFAR100-->CIFAR10 AUROC: {auroc_score(score_in, score_out):.2f}')

### Expected results

AUROC may slightly vary due to random initialization of linear probing.
```
CIFAR100-->CIFAR10 AUROC: 74.81
```

# Part VI. Mahalanobis distance as OOD score
- Use the output of the linear layer from task 4 as features to compute the Mahalanobis distance and the relative Mahalanobis distance.
- To compute the Mahalanobis distance group the features by their pseudolabels and compute the mean and covariance matrix for each class.

In [ ]:
### optional to use
@torch.no_grad()
def calc_maha_distance(embeds, means_c, inv_cov_c):
    ### START CODE HERE ### (≈ 3 lines of code)
    
    ### END CODE HERE ###
    return dist

def OOD_classifier_maha(train_embeds_in, train_labels_in, test_embeds_in, test_embeds_outs, num_classes,
                        relative=False):
    # optional to use our code!
    class_covs = []
    class_means = []
    used_classes = 0
    if type(train_labels_in) == torch.Tensor:
        train_labels_in = train_labels_in.cpu().numpy()
    if type(train_embeds_in) == torch.Tensor:
        train_embeds_in = train_embeds_in.cpu().numpy()
        test_embeds_in = test_embeds_in.cpu().numpy()
        test_embeds_outs = test_embeds_outs.cpu().numpy()
    ### START CODE HERE ### (≈ 23 lines of code)
    # calculate class-wise means and covariances
    
    # estimating the global std from train data
   
    # RMD: subtracting the average train score if relative is True
   
    # Get OOD score for each datapoint
    
    ### END CODE HERE ###
    return scores_in, scores_out

# The code below is provided based on our implementation. Optional to use!
num_classes = 100
lin_layer = load_model(lin_layer, "CLIP_best_max_train_acc.pth")
logits_indist_train, indist_pseudolabels_train = get_features(lin_layer, train_loader, device)
logits_indist_test, indist_pseudolabels_test = get_features(lin_layer, val_loader, device)
logits_ood, _ = get_features(lin_layer, ood_loader, device)
# convert to numpy
indist_pseudolabels_train = indist_pseudolabels_train.cpu().numpy()
indist_pseudolabels_test = indist_pseudolabels_test.cpu().numpy()
logits_indist_train = logits_indist_train.cpu().numpy()
logits_indist_test = logits_indist_test.cpu().numpy()
logits_ood = logits_ood.cpu().numpy()

# run OOD classifier based on mahalanobis distance
scores_in, scores_out = OOD_classifier_maha(logits_indist_train, indist_pseudolabels_train, 
                                            logits_indist_test, logits_ood, num_classes, relative=False)
print(f'Maha: CIFAR100-->CIFAR10 AUROC: {auroc_score(scores_in, scores_out):.2f}')
scores_in, scores_out = OOD_classifier_maha(logits_indist_train, indist_pseudolabels_train, 
                                            logits_indist_test, logits_ood, num_classes, relative=True)
print(f'Relative Maha: CIFAR100-->CIFAR10 AUROC: {auroc_score(scores_in, scores_out):.2f}')                                    

### Expected results
(can differ based on linear probing performance)

```
Maha: CIFAR100-->CIFAR10 AUROC: 83.31
Relative Maha: CIFAR100-->CIFAR10 AUROC: 80.88
```

# Part VII. Mahalanobis distance using the real labels without linear probing
- Again, compute the (relative) Mahalanobis distance as OOD score
- This time, instead of using the pseudolabels and output of the linear probing layer, use the real labels of the training data and the features computed in task 1

In [ ]:
### START CODE HERE ### (≈ 7 lines of code)
# load features

# load labels

# run OOD classifier based on mahalanobis distance

### END CODE HERE ###
print(f'Maha: CIFAR100-->CIFAR10 AUROC: {auroc_score(scores_md_in, scores_md_out):.2f}')
print(f'Relative Maha: CIFAR100-->CIFAR10 AUROC: {auroc_score(scores_rmd_in, scores_rmd_out):.2f}')

### Expected results
```
Maha: CIFAR100-->CIFAR10 AUROC: 71.71
Relative Maha: CIFAR100-->CIFAR10 AUROC: 84.93
```

# Part VIII. K-means clusters combined with Mahalanobis distance

The paper [SSD: A Unified Framework for Self-Supervised Outlier Detection](https://arxiv.org/abs/2103.12051) has proposed another unsupervised method for OOD detection. Instead of using the (real or pseudo) labels as class-wise means, we will now use the obtained clusters as found be kmeans. In more detail:

- Find k=10,50,100 clusters using Kmeans on the in-distribution training data (you can use the sklearn KMeans implementation).
- Get the cluster centers.
- Use them as class-wise means for the mahalanobis distance classifier.

In [ ]:
# The code below is provided based on our implementation. Optional to use!
# load features - modify names if you use different names
indist_train = torch.load('features/cifar100_train_feats.pt').cpu().numpy()
indist_test = torch.load('features/cifar100_test_feats.pt').cpu().numpy()
ood_test = torch.load('features/cifar10_test_feats.pt').cpu().numpy()
results_md = []
results_rmd = []
for N in [10,50,100]:
    ### START CODE HERE ### (≈ 7 lines of code)
   
    ### END CODE HERE ###
    print(f'Kmeans (k={N}) + RMD: CIFAR100-->CIFAR10 AUROC: {auroc_rmd:.2f}')
    print("-"*100)

### Expected results
Can differ based on KMenas performance.
```
Kmeans (k=10) + MD: CIFAR100-->CIFAR10 AUROC: 67.87
Kmeans (k=10) + RMD: CIFAR100-->CIFAR10 AUROC: 42.38
----------------------------------------------------------------------------------------------------
Kmeans (k=50) + MD: CIFAR100-->CIFAR10 AUROC: 72.18
Kmeans (k=50) + RMD: CIFAR100-->CIFAR10 AUROC: 58.73
----------------------------------------------------------------------------------------------------
Kmeans (k=100) + MD: CIFAR100-->CIFAR10 AUROC: 72.84
Kmeans (k=100) + RMD: CIFAR100-->CIFAR10 AUROC: 68.59
----------------------------------------------------------------------------------------------------
```

That's the end of this exercise. If you reached this point, **congratulations**!